In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
read_file = pd.read_excel ("Dataset - Penny test.xlsx")

In [3]:
read_file.to_csv ("Dataset_Penny_test.csv",index = None,header=True)

In [4]:
df = pd.DataFrame(pd.read_csv("Dataset_Penny_test.csv"))

In [5]:
df.head()

,SKU,Module,Origin,Origin_Arabic,Bar_Code,Category,Arabic_Category,Procpal category,Sub-Category,Arabic_Sub_Category,...,UoM,Arabic_UoM,Weight(kg),Length(cm),Width(cm),Height(cm),Brand,Arabic_Brand,Source,Identifier
0,ASTRO5225,5225,TAIWAN,تايوان,5225,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,PCS,حبة,1.10,24.0,17.0,5.3,Astro®,أسترو®,Astro Website,http://www.astrotools.com/air-tools/drills-scr...
1,ASTRO5275,5275,TAIWAN,تايوان,5275,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,PCS,حبة,1.80,22.5,17.0,5.5,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-extra-heavy-duty...
2,ASTRO1245,1245,TAIWAN,تايوان,745227007745,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,PCS,حبة,0.78,16.7,8.0,4.5,Astro®,أسترو®,Astro Website,http://www.astrotools.com/industrial-1-4-air-d...
3,ASTRO131,131AL,TAIWAN,تايوان,131AL,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,PCS,حبة,1.30,27.0,8.0,4.5,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-air-ratchet-wren...
4,ASTRO1812,1812,TAIWAN,تايوان,745227001903,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,PCS,حبة,2.83,21.3,19.8,7.7,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-super-duty-impac...


In [6]:
df1 = df['Identifier']
df1=df1[1:8]

# To Scrape Product_url

In [7]:
product_urls = []
for i in range(0,1):
    url=df['Identifier'].iloc[i].format(1)
    #print(url)
    req=requests.get(url)
    soup = BeautifulSoup(req.text, "html5lib")
    parent_class = soup.find_all("div",class_="dswc_item_wrapper swiper-slide")
    for purl in parent_class:
        product_url = purl.find("a")
        #print(product_url['href'])
        product_url = product_url['href']
        prod_url_dict = {'product_url':product_url}
        product_urls.append(prod_url_dict)  

# To Scrape Product_Details

In [8]:
astro_data=[]
product_urls_data = pd.DataFrame(product_urls)
for i in range(len(product_urls_data)):
    product_urll=product_urls_data['product_url'].iloc[i].format(1)
    request=requests.get(product_urll)
    soup = BeautifulSoup(request.text, "html5lib")
    parent = soup.find_all("div",{"class":"et_pb_row et_pb_row_1_tb_body"})
    for p in parent:
        product_name = p.find("h1")       
        if product_name:
            product_name=product_name.text
        else:
            product_name=None
            
        product_specs = p.find_all("div",{"class":"et_pb_tab_content"})[1]

        if product_specs:
            product_specs=product_specs.text.replace('\n','').replace('\t','')
        else:
            product_specs=None
            
        product_img = p.find("img",{"class":"wp-post-image"})

        if product_img:
            product_img=product_img['data-large_image']
        else:
            product_img=None
            
        sku = p.find("span",{"class":"sku"})

        if sku:
            sku=sku.text
        else:
            sku=None
            
        product_cat = p.find("span",{"class":"posted_in"})
        
        if product_cat:
            product_cat=product_cat.text
        else:
            product_cat=None
        
        source = {'source':'Astro Website',
                  'product_name':product_name,
                  'product_category':product_cat,
                  'SKU':sku,
                  'product_specifications':product_specs,
                  'product_url':product_urll,
                  'product_image_url':product_img
                 }
        astro_data.append(source)
            
        


In [9]:
Data=pd.DataFrame(astro_data)

In [10]:
Data

,source,product_name,product_category,SKU,product_specifications,product_url,product_image_url
0,Astro Website,3/8″ Nano Flex-Head Angle Impact Wrench – 400f...,"Categories: Air Tools, Impact Wrenches, New Pr...",1833,"• Square Drive: 3/8""• Working Torque: 280ft.lb...",https://www.astrotools.com/product/onyx-1-2-na...,https://www.astrotools.com/wp-content/uploads/...
1,Astro Website,1/2″ Nano Flex-Head Angle Impact Wrench,"Categories: Air Tools, Impact Wrenches, New Pr...",1834,,https://www.astrotools.com/product/1-2-nano-fl...,https://www.astrotools.com/wp-content/uploads/...
2,Astro Website,ONYX 1/2 Inch “THOR” Impact Wrench,"Categories: Air Tools, Impact Wrenches, New Pr...",1894,Max Torque: 940 ft-lbs (compare to 1600 nut-bu...,https://www.astrotools.com/product/onyx-1-2-in...,https://www.astrotools.com/wp-content/uploads/...
3,Astro Website,ONYX Flex Head Reversible HD Cut Off Tool,"Categories: Air Tools, Cutoff Tools",408,"• Wheel Size: 3"" (75mm)• Free Speed: 16,000 RP...",https://www.astrotools.com/product/onyx-flex-h...,https://www.astrotools.com/wp-content/uploads/...
4,Astro Website,ONYX Gear Driven Heavy Duty Air Saw,"Categories: Air Tools, Cutoff Tools",936,"Strokes per Minute: 7,000Ave. Air Consumption:...",https://www.astrotools.com/product/onyx-gear-d...,https://www.astrotools.com/wp-content/uploads/...
5,Astro Website,ONYX 3/8″ Nano Angle Impact Wrench – 415ft/lb,"Categories: Air Tools, Impact Wrenches, New Pr...",1838,"Square Drive: 3/8""Handle Type: Straight right ...",https://www.astrotools.com/product/onyx-3-8-na...,https://www.astrotools.com/wp-content/uploads/...


In [11]:
Data.to_csv('astro_product_details.csv',index=False)